In [71]:
import numpy as np
import pandas as pd
import os
import cv2


In [72]:
train=pd.read_csv('train.csv')
train.head()

,ID,Class
0,377.jpg,MIDDLE
1,17814.jpg,YOUNG
2,21283.jpg,MIDDLE
3,16496.jpg,YOUNG
4,4487.jpg,MIDDLE


In [73]:
test=pd.read_csv('test.csv')
test.head()

,ID
0,25321.jpg
1,989.jpg
2,19277.jpg
3,13093.jpg
4,5367.jpg


In [74]:
train.describe()

,ID,Class
count,19906,19906
unique,19906,3
top,5050.jpg,MIDDLE
freq,1,10804


In [75]:
def image_read(imglist,path):
    image_array=[]
    for i in imglist:
        image=cv2.imread(path+i)
        image=cv2.resize(image,(64,64))
        image_array.append(image)
    image_array=np.array(image_array)    
    return image_array

In [76]:
train_img_path="Train/"
imglist=train['ID']
#print(imglist)
X_train=image_read(imglist,train_img_path)

In [77]:
X_train.shape

(19906, 64, 64, 3)

In [78]:
test_img_path="Test/"
imglist1=test['ID']
X_test=image_read(imglist1,test_img_path)

In [79]:
X_test.shape

(6636, 64, 64, 3)

In [80]:
X_train=X_train/255

In [81]:
X_test=X_test/255

In [82]:
Y_train=train['Class']
Y_train = Y_train.map({'YOUNG': 0, 'MIDDLE': 1, 'OLD': 2})

In [83]:
from sklearn.preprocessing import LabelBinarizer 
label_binarizer = LabelBinarizer()
label_binarizer.fit(Y_train)

Y_train=label_binarizer.transform(Y_train)


In [84]:
from keras.layers import Input,InputLayer, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Sequential,Model
from keras.callbacks import ModelCheckpoint

In [85]:
inputShape=(64,64,3)


In [86]:
input = Input(inputShape)
#x = Conv2D(32,(3,3),strides = (1,1),name='conv_layer1')(xInput)

x = Conv2D(64,(3,3),strides = (1,1),name='layer_conv1',padding='same')(input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2,2),name='maxPool1')(x)
#x = Dropout(0.5)(x)

x = Conv2D(128,(3,3),strides = (1,1),name='layer_conv2')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2,2),name='maxPool2')(x)
#x = Dropout(0.5)(x)

x = Conv2D(128,(3,3),strides = (1,1),name='layer_conv3')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2,2),name='maxPool3')(x)
#x = Dropout(0.5)(x)

x = Conv2D(64,(3,3),strides = (1,1),name='conv4')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2,2),name='maxPool4')(x)


x = Flatten()(x)
x = Dense(64,activation = 'relu',name='fc0')(x)
x = Dropout(0.25)(x)
x = Dense(32,activation = 'relu',name='fc1')(x)
x = Dropout(0.25)(x)
x = Dense(3,activation = 'softmax',name='fc2')(x)

model = Model(inputs = input,outputs = x,name='Predict')

   

In [87]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 64, 64, 3)         0         
_________________________________________________________________
layer_conv1 (Conv2D)         (None, 64, 64, 64)        1792      
_________________________________________________________________
batch_normalization_54 (Batc (None, 64, 64, 64)        256       
_________________________________________________________________
activation_54 (Activation)   (None, 64, 64, 64)        0         
_________________________________________________________________
maxPool1 (MaxPooling2D)      (None, 32, 32, 64)        0         
_________________________________________________________________
layer_conv2 (Conv2D)         (None, 30, 30, 128)       73856     
_________________________________________________________________
batch_normalization_55 (Batc (None, 30, 30, 128)       512       
__________

In [88]:
model.save_weights('initial_weights.h5')

In [89]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [90]:
checkpointer = ModelCheckpoint(filepath='/weights.h5', verbose=1, save_best_only=True)
model.fit(X_train,Y_train,epochs = 40,batch_size = 128,callbacks=[checkpointer])

Epoch 1/40
19906/19906 [==============================] - 27s 1ms/step - loss: 0.8575 - acc: 0.6218
Epoch 2/40
  128/19906 [..............................] - ETA: 24s - loss: 0.7416 - acc: 0.7188

/opt/conda/lib/python3.6/site-packages/keras/callbacks.py:402: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


19906/19906 [==============================] - 25s 1ms/step - loss: 0.7419 - acc: 0.6825
Epoch 3/40
19906/19906 [==============================] - 25s 1ms/step - loss: 0.6674 - acc: 0.7126
Epoch 4/40
19906/19906 [==============================] - 25s 1ms/step - loss: 0.6151 - acc: 0.7386
Epoch 5/40
19906/19906 [==============================] - 25s 1ms/step - loss: 0.5792 - acc: 0.7572
Epoch 6/40
19906/19906 [==============================] - 25s 1ms/step - loss: 0.5398 - acc: 0.7741
Epoch 7/40
19906/19906 [==============================] - 25s 1ms/step - loss: 0.5029 - acc: 0.7915
Epoch 8/40
19906/19906 [==============================] - 25s 1ms/step - loss: 0.4626 - acc: 0.8099
Epoch 9/40
19906/19906 [==============================] - 25s 1ms/step - loss: 0.4305 - acc: 0.8234
Epoch 10/40
19906/19906 [==============================] - 25s 1ms/step - loss: 0.3999 - acc: 0.8406
Epoch 11/40
19906/19906 [==============================] - 25s 1ms/step - loss: 0.3711 - acc: 0.8523
Epoch 12/

In [93]:
pred=model.evaluate(X_train,Y_train)
print("Accuracy : " +str(pred[1]*100))
print("Total Loss  " +str(pred[0]*100))

19906/19906 [==============================] - 10s 504us/step
Accuracy : 98.3773736562
Total Loss  4.39766364003


In [95]:
model.save_weights('final_weight.h5')

In [96]:
predictions = model.predict(X_test)
predictions = np.argmax(predictions, axis= 1)

[1 0 0 1 1 1 1 0 1 1]


In [98]:
classes={0: 'YOUNG', 1: 'MIDDLE', 2: 'OLD'}

In [99]:
predicted_class=[classes[x] for x in predictions]

In [100]:
predicted_class[0:3]

['MIDDLE', 'YOUNG', 'YOUNG']

In [101]:
sub = pd.DataFrame({
    "Class": predicted_class,
    "ID": test['ID']
})

In [102]:
sub.to_csv("predicted_class.csv", index=False)